In [6]:
using JuMP 
using Gurobi
using LightGraphs
using DataFrames
using CSV
using TimerOutputs

myInstance = "2"

outfile = "/Users/din/Desktop/Research_S2019/Outputs/Tree_Clever_CVaR_"*myInstance*".txt"
gurobi_env = Gurobi.Env()

to = TimerOutput()
myFile = "myData"*myInstance*".jl"
include(myFile)
# include("myData10.jl")
setparams!(gurobi_env, Heuristics=0.0, Cuts = 0, OutputFlag = 0)


Academic license - for non-commercial use only


In [7]:

h1 = Model(solver = GurobiSolver(gurobi_env)) # If we want to add # in Gurobi, then we have to turn of Gurobi's own Cuts 

@variable(h1, 1 >= y_h[1:Len]>=0)
#@variable(h, q[1:Len]>=0)

#Setting constraint for start node
leaving = 1 .* (edge[j=1:Len,1] .== origin)
@constraint(h1, sum(y_h[k]*leaving[k] for k=1:Len) == 1)

#Setting constraint for other nodes

for i in all_nodes
    if i != destination
        if i != origin
            incoming = 1 .* (edge[j=1:Len,2] .== i)
            leaving = -1 .* (edge[j=1:Len,1] .== i)
            @constraint(h1, sum(y_h[k]*leaving[k] + y_h[k]*incoming[k] for k=1:Len) == 0)
        end
    end
end
#@objective(h, Min, sum((cL_orig[i]+d[i]*x_now[i])*y_h[i] + q[i] for i=1:Len))
# print(h1)
#solve(h1)



In [8]:
function what_arc(T, y, e)
    arc_type = -1 #type 0 = basic-SP, 1 = basic-nonSP, 2 = non-basic
    if T[e] == 1
        if y[e] == 1
            arc_type = 0
        else
            arc_type = 1
        end
    else
        arc_type = 2
    end
end

function classify_edges_costs(T,c)
    basic = []
    basic_cost = []
    i_basic = []
    nonbasic = []
    nonbasic_cost = []
    i_nonbasic = []
    
    for e = 1:Len
        #arc_type = what_arc(T,y,e)
        if T[e] != 1
            nonbasic = vcat(nonbasic, edge[e,:]')
            push!(i_nonbasic, e)
            push!(nonbasic_cost, c[e])
        else 
            basic = vcat(basic, edge[e,:]')
            push!(i_basic, e)
            push!(basic_cost, c[e])
        end
    end
    #We need to index back to the original edge values for reference purposes
    return basic, basic_cost, i_basic, nonbasic, nonbasic_cost, i_nonbasic
end




function find_affected_node(T, pred, e, edge) #NEED TO CONSIDER CYCLES
    S = Int[]
    N = Int[]
    push!(S, edge[e,2])
    push!(N, edge[e,2])
    visit = collect(1:last_node)
    l = length(S)
    visit[edge[e,2]] = 0
    while l > 0 
        node = S[1]
        for i = 1:last_node 
            if pred[i] == node && visit[i] != 0
                push!(S, i)
                push!(N, i)
                visit[i] = 0
            end
            
        end
        S[1], S[length(S)] = S[length(S)], S[1]
        pop!(S)
        l = length(S)
    end
    return N
end

function SA_basicSP(N, nonbasic, nonbasic_cost, label_temp, temp)
    delta = []
    delta_ind = []
    δ_ind = 0
    val_temp = 0
    if length(N) > 0
        ##println("nonbasic = ", nonbasic)
        for j in N
            head = find(nonbasic[:,2] .== j)
            if length(head) > 0
                ##println("head = ", head)
                for i in head
                    ##println("Arc #", i)
                    ##println("Arc name = ", nonbasic[i,:])
                    condd = nonbasic[i,1] in N
                    if condd == false
                        val_temp = label_temp[nonbasic[i,1]] + nonbasic_cost[i] - label_temp[nonbasic[i,2]]
                        push!(delta, val_temp)
                        push!(delta_ind, i)
                    end
                end
            end
        end
    end
    #IF DELTA DOESNT RETURN ANY VALUE AKA EMPTY: LET MIN DELTA = -1 THEN REMOVE IT LATER
    ###println("GOOD HERE")
    ##println("delta = ", delta)
    if length(delta) > 0
        for i = 1:length(delta)
            if i == 1
                δ_min = delta[1]
                δ_ind = delta_ind[1]
            end
            if delta[i] < δ_min
                δ_min = delta[i]
                δ_ind = delta_ind[i]
            end
        end
        temp = temp - δ_min
    else
        δ_min = -1.0
    end

    return δ_min, δ_ind, temp

end

function SA_basic(N, nonbasic, nonbasic_cost, label_temp, temp)
    delta = []
    delta_ind = []
    δ_ind = 0
    val_temp = 0
    if length(N) > 0
        ##println("nonbasic = ", nonbasic)
        for j in N
            tail = find(nonbasic[:,1] .== j)
            if length(tail) > 0
                ##println("tail = ", tail)
                for i in tail
                    ##println("Arc #", i)
                    ##println("Arc name = ", nonbasic[i,:])
                    condd = nonbasic[i,2] in N
                    if condd == false
                        ##println("FALSE THEN")
                        val_temp = label_temp[nonbasic[i,1]] + nonbasic_cost[i] - label_temp[nonbasic[i,2]]
                        push!(delta, val_temp)
                        push!(delta_ind, i)
                    end
                end
            end
        end
    end
    
    ##println("delta = ", delta)
    if length(delta) > 0
        ##println("DELTA > 0")
        for i = 1:length(delta)
            if i == 1
                δ_min = delta[1]
                δ_ind = delta_ind[1]
            end
            if delta[i] < δ_min
                δ_min = delta[i]
                δ_ind = delta_ind[i]
            end
        end
        temp = temp - δ_min
    else
        δ_min = -1.0
    end

    return δ_min, δ_ind, temp
end

SA_basic (generic function with 1 method)

In [9]:
function gx_bound(c_L, c_U, c, c_g, x_now, edge)
    
    #println(f,"Current CELL's LB = ", c_L)
    #println(f,"Current CELL's UB = ", c_U)
    #println(f, "Current interdiction x = ", x_now)
    start_node = edge[:,1]
    end_node = edge[:,2]

    no_node = max(maximum(start_node), maximum(end_node) )
    no_link = length(start_node)


    function getShortestX(state, start_node, end_node, origin, destination)
        _x = zeros(Int, length(start_node))
        _path = enumerate_paths(state, destination)

        for i=1:length(_path)-1
            _start = _path[i]
            _end = _path[i+1]

            for j=1:length(start_node)
                if start_node[j]==_start && end_node[j]==_end
                _x[j] = 1
                break
                end
            end

        end
        _x
    end


    graph = Graph(no_node)
    distmx = Inf*ones(no_node, no_node)

    # Adding links to the graph
    for i=1:no_link
        add_edge!(graph, start_node[i], end_node[i])
        distmx[start_node[i], end_node[i]] = c_g[i]
    end

    # Run Dijkstra's Algorithm from the origin node to all nodes
    state = dijkstra_shortest_paths(graph, origin, distmx)
    label = state.dists
    pred = state.parents
    b_arc = ""
    
    for i = 1: length(state.parents)
        if state.parents[i] != 0 
            b_arc = string(b_arc, "(", state.parents[i], ",", i, ")")
        end
    end
    
    # Retrieving the shortest path
    path = enumerate_paths(state, destination)
    
    #parents = LightGraphs.DijkstraState(state, destination)

    # Retrieving the 'x' variable in a 0-1 vector
    y = getShortestX(state, start_node, end_node, origin, destination)
    #println(f,"y vector:", y)
    
    gx = sum(c_g[i]*y[i] for i = 1:no_link)    
    SP = sum(c[i]*y[i] for i = 1:length(c))
    T = Int64[]
    for i = 1:Len
        if pred[edge[i,2]] == edge[i,1]
            push!(T, 1)
        else
            push!(T, 0)
        end
    end

    y_index = find(y .== 1)

    #println("Minimum Spanning Tree: ", T)
    #println("Shortest path y = ", y)
    #println("Indices of shortest path (edge) = ", y_index)
    #println("Nodes visited =", path)
    #println("Minimum Spanning Tree =", pred)
    #println("Node label =" ,label)

    return y, gx, SP, T, pred, label, path
end
function hx_bound(h, c_L, c_U, c, d, x_now, edge)
    
    c = (c_L + c_U)/2
    #println(f,"Current CELL's LB = ", c_L)
    #println(f,"Current CELL's UB = ", c_U)
    #println(f,"Current interdiction x = ", x_now)
    M = c_U - c_L

    h2 = copy(h1)

    y2 = h2[:y_h]

    @variable(h2, q[1:Len]>=0)
    for i = 1:Len
        @constraint(h2, q[i] >= c[i] - c_L[:,1][i] - M[i]*(1-y2[i])) #_h[i]))
    end

    @objective(h2, Min, sum((c_L[i]+d[i]*x_now[i])*y2[i] + q[i] for i=1:Len))

    #print(f,h2)
    solve(h2)
    hx = getobjectivevalue(h2)
    
    return getvalue(y2), hx
end

hx_bound (generic function with 1 method)

In [10]:
#MAIN PROGRAM:
f = open(outfile, "w")
df = DataFrame(CELL = Int[],ARCS_USED = Array[], LB = Array[], UB = Array[], PROB = Float64[], SP = Float64[])
df2 = DataFrame(ITER = Int[], X = Array[])
Cell_List = [1]
x_count = 0
weighted_unc = []
w_u = sum(cU_orig[i] - cL_orig[i] for i = 1:Len)
push!(weighted_unc, w_u)
weighted_unc_temp = copy(weighted_unc)
push!(df, (1, yy, cL_orig, cU_orig, 1, SP_init))
last_x = zeros(Len)
MP_obj = 0.0
##println(f,"MASTER PROBLEM==========================================================================================")

@timeit to "MP" begin 
m = Model(solver = GurobiSolver(gurobi_env)) # If we want to add # in Gurobi, then we have to turn of Gurobi's own Cuts 

@variable(m, w )
@variable(m,  x[1:Len], Bin)
@variable(m, 1e6 >= z[1:50000] >= 0)
@constraintref constr[1:50000]
@constraint(m, sum(x[i] for i=1:Len) <= 2.0) #attack budget = 2
constr[1] = @constraint(m, w <= SP_init + sum(yy[i]*x[i]*d[i] for i=1:Len) + z[1]  )
@objective(m, Max, α*w - sum(p[i]*z[i] for i = 1:length(p)) )#w - sum(s[k] for k=1:length(s))/length(s) )
##println(f,p)
##println(f,z[1:length(p)])

con_num = 1
stopping_cond = delta2 + 0
total_time = 0.0
iter = 0
@timeit to "BIGWHILE" while stopping_cond >= delta2 #&& length(df[:CELL]) <100  
        tic()
        stopping_cond = 0
        iter = iter + 1
        
#         println(f,"\n\nITERATION ",length(df[:CELL]),": ========================================================================================== ")
        ##println("Iteration: ", length(df[:CELL]))
        ###println("\nITERATION ", iter)
        ###println("Cell List = ", Cell_List)
        @timeit to "SOLVE M" begin
            solve(m) 
            end# END OF SOLVE M TIMING BLOCK
        
        cell_num = maximum(df[:CELL])
        MP = getvalue(α*w - sum(p[i]*z[i] for i = 1:length(p)))
        MP_obj = copy(MP)
        x_now = getvalue(x) + 0.0
#         println(f, "Interdiction ", x_now)
        
        ###println("x = ", x_now)
        push!(df2, (length(df[:CELL]), x_now))

        if length(df[:CELL]) > 1 #CHECK IF LAST INTERDICTION IS THE SAME
            if x_now == last_x
                x_count = x_count + 1
            else
                x_count = 1
                Cell_List = collect(1:length(p))
                weighted_unc_temp = copy(weighted_unc)
            end
        end
        ###println(f,"weighted_unc_temp = ", weighted_unc_temp)
        last_x = x_now
#         println(f, "\n No. ", x_count)
#         println(f,"Cell List: ", Cell_List)


        if x_count == 10
            ###println("\n\nSensitivity Analysis")
            iter = iter -1
            #reset x_count, last_x so next iteration won't build up on this SA
            x_count = 0
            #Do sensitivity analyis
            Delta_Final = zeros(Len)
            cell_num = 0
            k=0
            while k < length(Cell_List) #for cell_num=1:length(Cell_List)
                
                k = k + 1
                
                cell_num = Cell_List[k]

                row = find(df[:CELL] .== cell_num)[1]
                c_L = df[:LB][row] 
                c_U = df[:UB][row]   
                r_SA = 0.5*(c_U - c_L)
                c = (c_U + c_L)/2
                M = zeros(Len)
                for i = 1:Len
                    M[i] = c_U[i] - c_L[i]
                end
                c_g = c + d.*x_now
                
                #FIND TREE, PRED
                y, gx, SP, T, pred, label, path = gx_bound(c_L, c_U, c, c_g, x_now, edge)
                
                #println(f, "Current Cell ", k)
                ###println(f,"Cell_List ", Cell_List)
                ###println(f,"c_L ", c_L)
                ###println(f,"c_U ", c_U)
                
                
                #println(f, "g = ", g)
                #println(f, "g[k] = ", gx)
                g[cell_num] = gx
                
                
                
                #################################SENSITIVITY STARTS#################################
                #basic, basic_cost, i_basic, nonbasic, nonbasic_cost, i_nonbasic = classify_edges_costs(T,y,c_g)
            
                ###println("Shortest path edges: ", basicSP)
                ###println("Shortest path costs: ", basicSP_cost)
                ###println("Basic non-sp edges: ", basic)
                ###println("Basic non-sp costs: ", basic_cost)
                ###println("Non-basic edges: ", nonbasic)
                ###println("Non-basic costs: ", nonbasic_cost)
                
#                 println(f,"c_U = ", c_U)
#                 println(f,"c_L = ", c_L)
                
                for e = 1:Len
                    if r_SA[e] > 0
                        #taboo_list = []
                        basic, basic_cost, i_basic, nonbasic, nonbasic_cost, i_nonbasic = classify_edges_costs(T,c_g)
                        ##println("\n\nArc: ", edge[e,:])
                        ##println( "Current T = ", T)
                        ##println( "Current y = ", y)
                        ##println("Current label = ", label)
                        ##println("c = ", c_g)
                        label_temp = copy(label)
                        Δ = 0
                        δ = []
                        δ_ind = 0
                        T_temp = copy(T)
                        pred_temp = copy(pred)
                        c_temp = copy(c_g)
                        edge_num = copy(e)
                        N = Int[]
                        cur_edge = Int[]
                        path_temp = copy(path)
                        arc_type = what_arc(T_temp, y, e)
                        new_SP = false
                        ##println("Type: ", arc_type)
                        if arc_type == 2
                            u = edge[e,1]
                            v = edge[e,2]
                            t = pred[v]
                            e_temp = [t v]
                            B = find(all(basic .== e_temp,2)) #find a tree edge to be replaced by nonbasic e
                            i = B[1]
                            i = i_basic[i]
                            val = label_temp[u] - label_temp[v] + c_temp[e]
                            push!(δ, val)
                            δ_min = minimum(δ)
                            c_temp[e] = c_temp[e] - δ_min
                            Δ = Δ + δ_min
                            pred_temp[v] = u
                            T_temp[e] = 1
                            T_temp[i] = 0
                            if T_temp == T
                                new_SP = false
                            else
                                new_SP = true
                                Delta_Final[e] = Δ
                            end
                            arc_type = what_arc(T_temp, y, e)
                            
                        end
                        #loop = 0
                        while new_SP == false
                            basic, basic_cost, i_basic, nonbasic, nonbasic_cost, i_nonbasic = classify_edges_costs(T_temp,c_temp)
                            #loop = loop+1
                            ###println("Loop ", loop)
                            ##println( "Current T = ", T_temp)
                            ##println( "Current y = ", y)
                            ##println("Current label = ", label_temp)
                            
                            if arc_type == 0 #THIS STATEMENT IS WORKING
                                N = find_affected_node(T_temp, pred_temp, e, edge)
                                #Hereeeeee: Must update tree, path
                                ##println("N = ", N)
                                δ_min, δ_ind, temp = SA_basicSP(N, nonbasic, nonbasic_cost, label_temp, c_temp[e])
                                ##println("Δ = ", Δ)
                                ##println("δ_min = ", δ_min)
                                if δ_min < 0
                                    Delta_Final[e] = δ_min
                                    new_SP = true
                                else
                                    ##println("nonbasic arcs: ", nonbasic)
                                    ##println("index of arc moving into the basis: ", δ_ind)
                                    u = nonbasic[δ_ind, 1]
                                    v = nonbasic[δ_ind, 2]
                                    t = pred_temp[v]
                                    ##println("pred = ", pred_temp)
                                    ##println("u = ", u)
                                    ##println("v = ", v)
                                    ##println("t = ", t)
                                    e_temp = [t v]
                                    ##println("Basic arc becoming nonbasic: ", e_temp)
                                    #push!(taboo_list, i_nonbasic[δ_ind])
                                    ##println("Nonbasic arc becoming basic: ", [u v])
                                    ##println("basic arcs = ", basic)
                                    B = find(all(basic .== e_temp,2)) #find a tree edge to be replaced by nonbasic e
                                    ##println("B = ", B)
                                    i = B[1]
                                    i = i_basic[i]
                                    pred_temp[v] = nonbasic[δ_ind, 1]
                                    c_temp[e] = temp
                                    T_temp[i_nonbasic[δ_ind]] = 1
                                    T_temp[i] = 0 
                                    Δ = Δ + δ_min
                                    Delta_Final[e] = Δ
                                    ##println("CHECK MY CURRENT TREE: ", T_temp)
                                end
                            end
                            if arc_type == 1
                                N = find_affected_node(T_temp, pred_temp, e, edge)
                                ##println("N = ", N)
                                δ_min, δ_ind, temp = SA_basic(N, nonbasic, nonbasic_cost, label_temp, c_temp[e])
                                ##println("Δ = ", Δ)
                                ##println("δ_min = ", δ_min)
                                if δ_min < 0
                                    Delta_Final[e] = δ_min
                                    new_SP = true
                                else
                                    ##println("nonbasic arcs: ", nonbasic)
                                    ##println("index of arc moving into the basis: ", δ_ind)
                                    u = nonbasic[δ_ind, 1]
                                    v = nonbasic[δ_ind, 2]
                                    t = pred_temp[v]
                                    e_temp = [t v]
                                    ##println("Basic arc becoming nonbasic: ", e_temp)
                                    ##println("Nonbasic arc becoming basic: ", [u v])
                                    B = find(all(basic .== e_temp,2)) #find a tree edge to be replaced by nonbasic e
                                    i = B[1]
                                    i = i_basic[i]
                                    pred_temp[v] = nonbasic[δ_ind, 1]
                                    c_temp[e] = temp
                                    T_temp[i_nonbasic[δ_ind]] = 1
                                    T_temp[i] = 0 #####http://localhost:8889/notebooks/Desktop/Research_S2019/(C)Partition_Tree_Clever.ipynb############
                                    Δ = Δ + δ_min
                                    Delta_Final[e] = Δ
                                    ##println("Pred = ", pred_temp)
                                end
                                
                            end 
                            #CHECK FOR LOOP:
                            if T_temp != T
                                new_SP = true
                            else
                                loop = false
                                if pred_temp[origin] != 0
                                    loop = true
                                else
                                    v = edge[e,2]
                                    visit = collect(1:last_node)
                                    ##println("visit = ", visit)
                                    while v != origin && visit[v] != 0
                                        ##println("v = ", v)
                                        ##println("pred[v] = ", pred_temp[v] )
                                        visit[v] = 0
                                        v = pred_temp[v]
                                        
                                        ##println("visit = ", visit)
                                        
                                    end
                                    
                                    if v != origin
                                        loop = true
                                        ##println("HERE")
                                    end
                                end
                                if loop == true
                                    new_SP = true
                                    
                                end
                            end
                            ##println("new_SP = ", new_SP)
                            
                        end
                    else
                        Delta_Final[e] = -1.0
                    end
                end
                
                rule100 = 0
                  
                rule100 = sum(r_SA[e]/Delta_Final[e] for e = 1:length(Delta_Final) if Delta_Final[e] != -1.0) 
                ###println(f,"r_SA = ", r_SA)
                ###println(f,"Delta_Final = ", Delta_Final)
#                 println(f, "100% Rule Result for cell ", cell_num, ": ", rule100)
                
                if rule100 <= 1.0 
                    y_h, hx = hx_bound(h, c_L, c_U, c, d, x_now, edge) 
                    h[cell_num] = hx
                    Cell_List[k], Cell_List[length(Cell_List)] = Cell_List[length(Cell_List)], Cell_List[k]
                    pop!(Cell_List)
                    k = k - 1
                                
                                
                                ####TESTING IN PROGRESSS============
#                     y_h, hx = hx_bound(h, c_L, c_U, c, d, x_now, edge)
#                     if length(h) >= k 
#                         h[cell_num] = hx
#                     else
#                         push!(h,hx)
#                     end
                                ###END OF TESTING=============
                                
                                
                end
                            
#                 println(f,"Cells Remaining: ", Cell_List, " \n\n")
                stopping_cond = delta2 + 1
                #################################SENSITIVITY ENDS#################################
            end
            
            Cell_List = sort(Cell_List)
        else
            
            #Do the normal thing
            z_now = getvalue(z)+ 0.0 
            ##println(f,"z = ", z_now[1:length(p)])
            ###println("Cell List = ", Cell_List)
            c_L = cL_orig + 0.0
            c_U = cU_orig + 0.0
            c = (c_L + c_U)/2
            y = zeros(length(c_L))
            r = 0
            r2 = 0
            gap = 0.0
            
            t = 0
            k = 0
            found = false
                
            myLength = length(weighted_unc_temp[weighted_unc_temp.>0])#length(Cell_List)
            ###println("myLength = ", myLength)
            while found == false && t < myLength #length(Cell_List)# k = 1: cell_num #length(p)
            #while t < myLength
                    
                    #println(f,"\nWeighted_unc_temp ", weighted_unc_temp)
                    #println(f,"Length of Cell_List = ", length(Cell_List))
                    @timeit to "Pre-gx" begin
                    t = t + 1
#                     if iter >=20
#                         println("t = ", t)
#                         println("weighted = ", weighted_unc_temp)
#                     end
                    found_max = false
                    while found_max == false            
                        max_val = maximum(weighted_unc_temp)
                        Max_List = find(weighted_unc_temp .== max_val)
                        ind_k = Max_List[1]
                        #k = Cell_List[ind_k]
                                    #println("ind_k = ", ind_k)
                                    #println("Cell_List = ", Cell_List)
                        in_list = ind_k in Cell_List
                                    #println("in_list? ", in_list)
                        if in_list == true
                            k = ind_k
                            found_max = true
                        else
                            weighted_unc_temp[ind_k] = 0.0 
                        end
                        
                    end           
                    #println("ind_k", ind_k)
                    #k = Cell_List[ind_k]
                    #k=k+1
                    #println(f,"\nCURRENT CELL = ", k)
                    
                    if length(find(df[:CELL] .== k))>0
                        row = find(df[:CELL] .== k)[1]
                        c_L = df[:LB][row] 
                        c_U = df[:UB][row] 
                    else
                        c_L = cL_orig[:,1] 
                        c_U = cU_orig[:,1] 
                    end
                    c = (c_U + c_L)/2
                    M = zeros(Len)
                    for i = 1:Len
                        M[i] = c_U[i] - c_L[i]
                    end
                    c_g = c + d.*x_now
                    end # END OF Pre-gx TIMING BLOCK
                ##println(f,"==================SUBPROBLEM 1: G(X), CURRENT CELL = ", k)
                ###println("c_U = ", c_U )
                ###println("c_L = ", c_L )
                            
                @timeit to "gx" begin
                    y, gx, SP, T, pred, label, path = gx_bound(c_L, c_U, c, c_g, x_now, edge)
#                     println("z[k] = ", getvalue(z[k]))
#                     println("gx[k] = ", gx)
                    
                    ###println("g = ", g)
#                     println(f,"Current Cell = ", k)
                    if length(g) >= k 
                        g[k] = gx
                    else
                        push!(g,gx)
                        push!(h, 0.0)
                    end  
                    end #END OF GX TIMING BLOCK
                
                if (z_now[k] + gx) < getvalue(w) - delta1
                    @timeit to "Add Constraints (1)" begin
#                         println(f,"z = ", z_now[k])
#                         println(f,"g = ", gx)
                        stopping_cond = delta2 + 1
                        r = k 
                        ##println(f,"k = ", k)
                        push!(df, (r, y, c_L + zeros(length(c_L)), c_U + zeros(length(c_U)), p[r], SP))
                        
                        last_row = length(df[:CELL])
                        con_num = con_num + 1 
                        constr[last_row] = @constraint(m, w <= z[r] + sum(c[i]*y[i] + d[i]*y[i]*x[i] for i = 1:length(c_L)))
                        ##println(f,"Constraint #", last_row," is :", constr[last_row])
                        @objective(m, Max, α*w - sum(p[i]*z[i] for i = 1:length(p)) )

                        found = true
                        ###println(f,"Length of Cell_List = ", length(Cell_List))
                        ###println(f,"Cell List ", Cell_List)
                        ##println(f,"FOUND = ", found)
                        ##println(f,"FOR CELL ", k, ", STOPPING CONDITION (after gx): ", stopping_cond)
                        end #END OF CONSTRAINT 1 TIMING BLOCK
                else
                    ##println(f,"==================SUBPROBLEM 2: H(X), CURRENT CELL = ", k)
                    @timeit to "hx" begin
                        y_h, hx = hx_bound(h, c_L, c_U, c, d, x_now, edge)
                        #println(f,"h[k] = ", hx)
                        if length(h) >= k 
                            h[k] = hx
                        else
                            push!(h,hx)
                        end
                        end #END OF HX TIMING BLOCK
                    
                                
#                     println(f,"g = ", gx)
#                     println(f,"h = ", hx)
                    if gx-hx > delta2
                        @timeit to "Add Constraints (2)" begin    
                            stopping_cond = delta2 + 1
                           
                            found = true
                            
                            arc_split = find(M .== maximum(M))[1]
                            gap = (c_U[arc_split] - c[arc_split])/2
                            r2 = maximum(df[:CELL])+1
                                        
                            weighted_unc[k] = weighted_unc[k] - gap*(p[k]/2)
                            push!(weighted_unc, weighted_unc[k])
                                        
                            weighted_unc_temp[k] = weighted_unc[k]
                            push!(weighted_unc_temp, weighted_unc_temp[k])
                                        
                            push!(df, (r2, y, df[:LB][row]+zeros(length(c_L)), df[:UB][row]+zeros(length(c_L)), p[k]/2, SP))
                            push!(p,p[k]/2)
                            push!(Cell_List, r2)
                            push!(g,SP)
                            push!(h, 0.0)
             
                            
                            p[k]=p[k]/2
                            last_row = length(df[:CELL])
                            df[:LB][last_row][arc_split] = c[arc_split] + 0.0
                            c_temp = (df[:UB][last_row] + df[:LB][last_row]) / 2
                            y_temp = df[:ARCS_USED][last_row]
                            SP = sum(c_temp[t]*y_temp[t] for t=1:Len)
                            df[:SP][last_row] = SP

                            for i = 1: length(df[:CELL])-1
                                if df[:CELL][i] == k 
                                    df[:PROB][i] = df[:PROB][i]/2
                                    df[:UB][i][arc_split] = c[arc_split]+ 0.0

                                    #RE-CALCULATING COST OF COPIED SHORTEST PATH
                                    c_temp = (df[:UB][i] + df[:LB][i]) / 2
                                    y_temp = df[:ARCS_USED][i]
                                    SP = sum(c_temp[t]*y_temp[t] for t=1:Len)
                                    df[:SP][i] = SP
                                    JuMP.setRHS(constr[i], df[:SP][i])
                                end
                            end
                            constr[last_row] = @constraint(m, w - z[df[:CELL][last_row]] <=   sum(d[i]*x[i]*y[i] for i = 1:length(c_L)) + df[:SP][last_row]) 
                            @objective(m, Max, α*w - sum(p[i]*z[i] for i = 1:length(p)) )
                            end #END OF CONSTRAINT 2 TIMING BLOCK
                    else
                        weighted_unc_temp[k] = 0.0
                    end
                    ##println(f,"FOUND = ", found)
                    ##println(f,"FOR CELL ", k, ", STOPPING CONDITION (after adding constraints) : ", stopping_cond)
#                     println(f, "g = ", g)
#                     println(f, "h = ", h)
                end
            #@timeit to "WRITE TO FILE" CSV.write("/Users/din/Desktop/Research/OUTPUT(Test5Nodes_Mod).cvs", df)  
            end
            end
            
        time_lapse = toq()    
        ##println(f, "Elapsed Time for Iteration ", k," = ", time_lapse)
        total_time = total_time + time_lapse
        ##println(f,"Total Running Time = ", total_time)
        ###println("Total Running Time = ", total_time)
    end
    end #END OF MP TIMING BLOCK    

##CSV.write("/Users/din/Desktop/Research/Cell_CellTest_SA.cvs", df)  
##CSV.write("/Users/din/Desktop/Research/InterdicionX_CellTest_SA", df2)  

@timeit to "CALC OPT GAP" opt_gap = sum(p[i]*(g[i] - h[i]) for i = 1:length(p))
##println(f,"\n\nweighted_unc_temp = ", weighted_unc_temp)
##println(f,"\n\n\nFinal z = ", getvalue(z[1:length(p)]))
##println(f,"Final g = ", g)
##println(f,"Final h = ", h)
##println(f,"p = ", p)
##println(f,"Optimality gap = ", opt_gap)
##println(f,"List of all cells = ", Cell_List)

println(f,"Optimality gap = ", opt_gap)
println(f,"MP_obj = ", MP_obj)
println(f,"Overall runtime = ", total_time)
println(f,"No. iterations = ", length(df[:CELL]))
# for i = 1 : length(g)
#     println(i, ". g = ",g[i],", h = ", h[i])
# end

close(f)

show(to; allocations = false)

 ────────────────────────────────────────────────────────
                                           Time          
                                   ──────────────────────
         Tot / % measured:              75.7s / 98.4%    

 Section                   ncalls     time   %tot     avg
 ────────────────────────────────────────────────────────
 MP                             1    74.5s   100%   74.5s
   BIGWHILE                     1    74.4s   100%   74.4s
     SOLVE M                  249    20.8s  27.9%  83.5ms
     hx                       399    331ms  0.44%   829μs
     gx                       485    262ms  0.35%   541μs
     Pre-gx                   485    145ms  0.20%   300μs
     Add Constraints (2)      140   76.2ms  0.10%   544μs
     Add Constraints (1)       86   6.86ms  0.01%  79.7μs
 CALC OPT GAP                   1   32.1ms  0.04%  32.1ms
 ────────────────────────────────────────────────────────